In [8]:
import tensorflow as tf
import pandas as pd
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model
# from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add, Input, Flatten

# import time
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
# import matplotlib.pyplot as plt
# import matplotlib.style as style
# import seaborn as sns
# import cv2
# from utils import *

# BATCH_SIZE = 256  # Big enough to measure an F1-score
# # Adapt preprocessing and prefetching dynamically
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# SHUFFLE_BUFFER_SIZE = 8192


In [15]:
dir_path = './database_test'

file_list = os.listdir(dir_path)
ID = []
Labels = []

for file_name in file_list:
    file_name.encode('ISO-8859-1')
    label_list = file_name.split('+')
    if (label_list[0] == ''):
        continue
    ID.append(label_list[0])
    del label_list[0]
    # .jpg 제거
    label_list.pop()

    valid_label_list = []
    for label in label_list:
        label = label.lower()
        labels = label.split(', ')
        valid_label_list.extend(labels)

    while True:
        try:
            valid_label_list.remove('')
        except ValueError:
            break

    Labels.append(valid_label_list)

artworks = pd.DataFrame({"ID": ID, "Labels": Labels, "File": file_list})
artworks.dropna()

# Get label frequencies in descending order
label_freq = artworks['Labels'].apply(
    lambda s: [x for x in s]).explode().value_counts().sort_values(ascending=False)


In [5]:
# Create a list of rare labels
# rare = list(label_freq[label_freq < 50].index)
rare = list(label_freq[label_freq < 25].index)
print("Number of used labels:", len(label_freq) - len(rare))
print("Number of ignored labels:", len(rare))
label_number = len(label_freq) - len(rare)

artworks['Labels'] = artworks['Labels'].apply(
    lambda s: [x for x in s if x not in rare])
artworks['Labels'] = artworks['Labels'].apply(
    lambda s: s if len(s) != 0 else np.nan)

artworks.dropna(inplace=True)
artworks.head(10)


Number of used labels: 42
Number of ignored labels: 36


,ID,Labels,File
0,101190,"[figurative, conceptual art]",101190+figurative+Conceptual Art+.jpg
1,101213,"[illustration, art nouveau (modern)]",101213+illustration+Art Nouveau (Modern)+.jpg
2,100831,"[illustration, expressionism]","100831+illustration+Art Deco, Expressionism+.jpg"
3,101834,"[flower painting, ink and wash painting]",101834+flower painting+Ink and wash painting+.jpg
4,100470,"[abstract, abstract expressionism]",100470+abstract+Abstract Expressionism+.jpg
5,10187,[northern renaissance],10187+mythological painting+Northern Renaissan...
6,101857,"[still life, expressionism]",101857+still life+Expressionism+.jpg
7,1008,"[religious painting, baroque]","1008+religious painting+Baroque, Tenebrism+oil..."
8,101812,"[still life, expressionism]",101812+still life+Expressionism+.jpg
9,101171,"[abstract, abstract art]",101171+abstract+Abstract Art+.jpg


In [7]:
from sklearn.model_selection import train_test_split
# 파일 경로 및 label 집합으로 train data와 valid data parsing
X_train, X_val, y_train, y_val = train_test_split(
    artworks['File'], artworks['Labels'], test_size=0.1, random_state=23)

X_train = [os.path.join('./database_test', str(f)) for f in X_train]
X_val = [os.path.join('./database_test', str(f)) for f in X_val]
X_train[:3]


['./database_test/100600+photo+Abstract Expressionism+.jpg',
 './database_test/101506+genre painting+Expressionism+.jpg',
 './database_test/100+still life+Naïve Art (Primitivism)+oil, copper.jpg']

In [9]:
# MultiLabelBinarizer로 다중 label 이진화
mlb = MultiLabelBinarizer()
mlb.fit(artworks['Labels'])

# Loop over all labels and show them
N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

y_train_bin = mlb.transform(y_train)
mlb.fit(y_val)
y_val_bin = mlb.transform(y_val)


0. abstract
1. abstract art
2. abstract expressionism
3. animal painting
4. art informel
5. art nouveau (modern)
6. baroque
7. cityscape
8. color field painting
9. conceptual art
10. concretism
11. cubism
12. expressionism
13. figurative
14. flower painting
15. genre painting
16. hard edge painting
17. illustration
18. impressionism
19. ink and wash painting
20. installation
21. landscape
22. lyrical abstraction
23. minimalism
24. nanga (bunjinga)
25. naïve art (primitivism)
26. neo-impressionism
27. northern renaissance
28. nude painting (nu)
29. op art
30. photo
31. pointillism
32. pop art
33. portrait
34. post-impressionism
35. realism
36. regionalism
37. religious painting
38. romanticism
39. still life
40. surrealism
41. symbolic painting


for _ 는 변수없이 반복문 수행

In [38]:
from itertools import combinations

node_df = pd.DataFrame(columns=['class'])
for (i, label) in enumerate(mlb.classes_):
  node_df.loc[i] = label

node_df.reset_index()

class_combinations = list(combinations(mlb.classes_, 2))
edge_df = pd.DataFrame(columns=['class1', 'class2', 'correlation'])

for (i, class_combination) in enumerate(class_combinations):
  edge_df.loc[i] = [class_combination[0], class_combination[1], 0]

edge_df.reset_index()

for label in Labels:
  label_combinations = list(combinations(label, 2))
  for label_combination in label_combinations:
    condition1 = (edge_df['class1'] == label_combination[0]) & (edge_df['class2'] == label_combination[1])
    condition2 = (edge_df['class1'] == label_combination[1]) & (edge_df['class2'] == label_combination[0])
    if (edge_df[(condition1) | (condition2)].index.tolist()):
      print(edge_df[(condition1) | (condition2)].index.tolist())
      edge
      
    # edge_df.index((edge_df['class1'] == label_combination[0] & edge_df['class2'] == label_combination[1]) | (edge_df['class2'] == label_combination[0] & edge_df['class1'] == label_combination[1]))


[336]
[206]
[430]
[487]
[1]
[452]
[261]
[452]
[0]
[31]
[15]
[269]
[270]
[396]
[22]
[22]
[15]
[1]
[206]
[663]
[532]
[206]
[15]
[653]
[829]
[206]
[206]
[663]
[3]
[853]
[296]
[28]
[487]
[22]
[1]
[15]
[3]
[653]
[649]
[206]
[0]
[206]
[761]
[9]
[206]
[257]
[434]
[609]
[432]
[3]
[9]
[446]
[3]
[1]
[22]
[92]
[487]
[97]
[426]
[1]
[9]
[206]
[263]
[261]
[22]
[3]
[399]
[206]
[428]
[512]
[587]
[206]
[1]
[343]
[128]
[135]
[432]
[3]
[206]
[1]
[825]
[166]
[128]
[135]
[432]
[432]
[292]
[128]
[108]
[3]
[1]
[257]
[206]
[239]
[261]
[853]
[128]
[135]
[432]
[660]
[655]
[745]
[667]
[206]
[653]
[3]
[426]
[257]
[9]
[434]
[206]
[9]
[9]
[7]
[1]
[1]
[28]
[15]
[15]
[9]
[108]
[502]
[860]
[434]
[426]
[206]
[0]
[398]
[15]
[736]
[128]
[135]
[432]
[649]
[587]
[108]
[427]
[487]
[432]
[92]
[487]
[97]
[31]
[518]
[343]
[1]
[860]
[1]
[1]
[15]
[0]
[206]
[587]
[667]
[502]
[1]
[1]
[1]
[343]
[257]
[15]
[28]
[548]
[860]
[206]
[669]
[28]
[860]
[587]
[0]
[829]
[1]
[1]
[525]
[520]
[745]
[128]
[206]
[22]
[353]
[21]
[851]
[825]
[9]
[2

In [ ]:
adj =[[0 for _ in range(N_LABELS)] for _ in range(N_LABELS)]
for i in range(N_LABELS):
  adj[i][i] = len(y_train_bin)

for label_list in y_train_bin:
  valid_label_list = []
  for idx, label in enumerate(label_list):
    if label == 1:
      valid_label_list.append(idx)
  for i in valid_label_list:
    for j in valid_label_list:
      if i != j:
        adj[i][j] += 1


print(adj)


In [ ]:
import tensorflow_gnn as tfgnn

graph_tensor = tfgnn.GraphTensor.from_pieces(
    node_sets= {
    
    }
)